In [ ]:
# 必要なライブラリのインストール
!apt install libzbar0
!pip install pyzbar

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libzbar0 is already the newest version (0.10+doc-10.1build2).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.


In [ ]:
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
import csv
import datetime

In [ ]:
from pyzbar.pyzbar import decode

#サポートパッチのインポート
from google.colab.patches import cv2_imshow
# cv2_imshow(image)

In [ ]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

In [ ]:
# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [ ]:
import math

def contrast(image, a):
    lut = [ np.uint8(255.0 / (1 + math.exp(-a * (i - 128.) / 255.))) for i in range(256)]
    result_image = np.array( [ lut[value] for value in image.flat], dtype=np.uint8 )
    result_image = result_image.reshape(image.shape)
    return result_image

In [ ]:
# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
count = 0 
data_pre =[] # 読んだバーコードの履歴
pre_bar = ''

BarcodeResultFile = 'Barcode reader result.csv'

dt_now = datetime.datetime.now()
with open(BarcodeResultFile, 'a') as f:
    writer = csv.writer(f, lineterminator='\n') # 改行コード指定
    writer.writerow(['Barcode start : '+str(dt_now)])

while True:
    # 処理時間測定用 1 of 2
    start_time = time.time()

    # print('I am before video_frame')
    js_reply = video_frame(label_html, bbox)
    # print('I am after video_frame')
    if not js_reply:
        break

    # convert JS response to OpenCV Image
    img = js_to_image(js_reply["img"])

    # create transparent overlay for bounding box
    bbox_array = np.zeros([480,640,4], dtype=np.uint8)

    # GrayScale
    imageGlay = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = contrast(imageGlay, 5)
    
    data = decode(img)

    # コード内容を出力
    for lc in range(len(data)):
        count = 0
        # Barcodeの枠を描画する
        pts = np.array([data[lc].polygon],np.int32)
        pts = pts.reshape((-1,1,2))
        cv2.polylines(bbox_array,[pts],True,(0,0,255),5)

        # Barcodeのデータを処理する
        barcode_data = data[lc][0].decode('utf-8', 'ignore')

        # Barcodeのデータを描画する
        bbox_array = cv2.putText(bbox_array,barcode_data,(50,50+lc*70),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,0,0),2)

        result = barcode_data not in data_pre
        if ( result ):
            pre_bar = barcode_data
            data_pre.append(barcode_data)
            with open(BarcodeResultFile, 'a') as f:
                writer = csv.writer(f, lineterminator='\n') 
                writer.writerow([barcode_data])
                # writer.writerow([[barcode_data]])
                
        else: 
            if barcode_data == pre_bar:
                bbox_array = cv2.putText(bbox_array,'OK! Bring next barcode',(150,240),cv2.FONT_HERSHEY_SIMPLEX,1.0,(255,0,0),3)
            else:
                bbox_array = cv2.putText(bbox_array,'Duplicate. Read alredy',(200,80+lc*70),cv2.FONT_HERSHEY_SIMPLEX,1.0,(255,0,0),3)

    # 処理時間の表示 2 of 2
    pro_time = time.time() - start_time
    time_lc = ' : processing time: {0:.2f}'.format(pro_time)
    bbox_array = cv2.putText(bbox_array,str(count)+time_lc,(50,430),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,0,0),2)
    count += 1

    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    # convert overlay of bbox into bytes
    bbox_bytes = bbox_to_bytes(bbox_array)

    # update bbox so next frame gets new overlay
    bbox = bbox_bytes



<IPython.core.display.Javascript object>

<font color='red'>Enjoy!</font> JMB
